# Using HF (Hugging Face) based datasets with infctx trainer

The infctx trainer makes a huge shift towards a HF focus dataset parser, with several pros and cons. This note book aims to cover all the common use cases for dataset handling and processing, that is supported by this trainer code.

As this guide is focused only on the dataset configuration option side of things. And all the examples here will not perform any real training / checkpointing of an output model

## Intial setup

Before we go into the dataset setup, lets perform an initial setup for all the folders we need, and a small toy model which we would use throughout the various examples within this notebook.

In [1]:
# Setup the folders we will need
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/

# Initialized a simple L6-D512 model, for both the v4 neox (50277) tokenizer
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size neox --skip-if-exists ../model/L6-D512-neox-init.pth

# and rwkv world (65529) tokenizers
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size world --skip-if-exists ../model/L6-D512-world-init.pth

# If you have a custom vocab size, you can indicate accordingly as well with an int
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size 20259 --skip-if-exists ../model/L6-D512-V20259-init.pth

[2023-08-04 11:23:28,641] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-neox-init.pth
Vocab size: 50277
---- ----- ----
Model exists, skipping init_model
[2023-08-04 11:23:31,917] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-world-init.pth
Vocab size: 65529
---- ----- ----
Model exists, skipping init_model
[2023-08-04 11:23:35,233] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
---- Initializing model ----
No of layers: 6
Embedding size: 512
O

## Training using a text dataset via Hugging Face

The following is the `mini-enwiki.yaml` settings, for using a textual dataset via huggingface, with most of the comments removed

```.yaml
trainer:
  max_steps: 10
  target_batch_size: 32
model:
  load_model: ../model/L6-D512-neox-init.pth
  ctx_len: 1024
  lr_init: 3e-4
  bptt_learning: true
  bptt_learning_range: -1
data:
  data_path: ../datapath/enwiki_10k_neox_1024/

  source: "teven/enwiki_10k"
  # # Additional source dataset params, used to grab subsets of the dataset
  # source_dataset_params:
  #   language: en

  tokenizer: neox
  min_token_size: 64
  max_token_size: -1
  text_rechunk_size: 2048
  text_rechunk_force: true
  custom_text_key: 'text'
  test_split: 0.01
  test_split_shuffle: false
```

### Understanding the `data` config, for textual datasets

Lets go through each of data parameter settings and what they mean...

**data.data_path** 
This is where the HF datapath is saved, when used against existing HF data sources.

**data.source** 
This can be configured either as a hugging face dataset (eg. `teven/enwiki_10k`) or if intended to be used with local files `text / json / csv / pandas` or their respective file paths (see the local file examples for more details) 

**data.source_dataset_params** 
Additional params to configure the huggingface `load_dataset` command. This is useful mostly for larger dataset which supports parameters, to filter out a subsets specifically (defaults to empty object)

**data.tokenizer**
The tokenizer to use for the dataset, use either `neox` or `world` for the respective RWKV models. For custom HF tokenizer refer to the custom tokenizer example. (defaults to neox)

**data.min_token_size/max_token_size**
Scans the given dataset, and skips datasamples that fail to meet the given criteria. This is mostly useful for filtering small low quality datasamples in large datasets, or large datasample beyond what you intend to support. (this is done before rechunking if enabled, defaults to -1 which support all)

**data.text_rechunk_force** : Enable text rechunking, this means, all the filtered datasamples will be merged together, with a new line between them. Before being split again by the rechunk size. This is mostly useful for large corpus of raw text data, and is consistent with how existing foundation model are trained from raw text files. This also allows more efficient training process (tokens/second), as each datasample will have the exact same token count. (Disabled by default unless your source is literally a ".txt" file)

**data.text_rechunk_size** : Number of tokens each datasample should have after rechunking. Recommended sizes is the context size you intend the model to support (ie. 2048, 4096, etc)

**data.custom_text_key** : For huggingface datasets (or json/csv) by default we would use the `text` collumn if its avaliable. However some dataset store their text in a different collumn (eg. `code`). This allow you to choose which collumn would you like to use the text from. Note for more complicated instruct/input/output examples, you will want to see the 'multi_column' guide/examples instead.

### Optimizing the `model.bptt_*` mode according to your dataset config ....

**model.load_model** 
This is the `model.pth` file you start the training process from. This can be an exisitng model you are finetuning from, or a new model that you initalized with `init_model.py` script.

**model.ctx_len** 
This is the training context length used in the training process. For the infctx trainer, your data samples can be larger then the training context length. If so, the data sample is split into chunks accordingly, and trained in parts (with bptt_learning enabled, which it is by default)

This is ultimately a tradeoff between VRAM usage vs GPU compute usage, while you can save VRAM usage, this comes an increased compute cost, as first few chunks will need to recalculated multiple times for each subsequent chunk. There are also been recorded minor loss learning penalty especially for small context sizes.

As such it is always recommended to configure this to be as large as what can be supported by your GPU in the power of 2 (1024,2048,4096,...) with some healthy vram buffer for checkpoints and gradients, and up to your dataset sample size (as its pointless to go beyond that)

Typically this is 4096, or 8192 for ML training GPUS (24GB vram and above). For consumer GPUS, anything less then 512 is not recommended, due to compounded loss learning penalty involved when used with large data samples.

**model.bptt_learning**
Enabled by default, this is the core feature of infctx trainer. If your training ctx length is equal to your dataset context length, you can disable bptt_learning for an insignificant speed boost (barely measurable).

In most cases its better to just set bptt_learning_range to 1 instead of switching it off

**model.bptt_learning_range**
While the default setting of `bptt_learning: true` and `bptt_learning_range: -1` will work by default for all use cases.

When training across multiple GPUs, `bptt_learning_range: -1` has a small performance penalty in which it needs to syncronize the 

### Download and preload the datapath from huggingface

In [3]:
# Lets preload the requried dataset
!cd ../../RWKV-v4neo && python3 preload_datapath.py ../notebook/dataset-config/mini-enwiki.yaml

[2023-08-04 11:31:18,816] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1022.75it/s]


In [2]:
# Validate source code and env is working, by doing a short 2 sample dryrun
!cd ../../RWKV-v4neo && python3 lightning_trainer.py fit -c ../notebook/dataset-config/mini-enwiki.yaml

[2023-08-04 11:27:20,394] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '../notebook/dataset-config/mini-enwiki.yaml'], args=['fit', '-c', '../notebook/dataset-config/mini-enwiki.yaml'].
  rank_zero_warn(
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 4139054562
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 4139054562
[RWKV.model]: Preloading model from '../model/L6-D512-world-init.pth'
Using /home/